In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [23]:
dataset = pd.read_csv('37639x144_samples.csv', delimiter=',')
dataset.head()

,game_time,score,net_worth,team_2_player_0_level,team_2_player_0_kill_count,team_2_player_0_death_count,team_2_player_0_assists_count,team_2_player_0_denies_count,team_2_player_0_lh_count,team_2_player_0_gold,...,team_3_player_4_lh_count,team_3_player_4_gold,team_3_player_4_net_worth,team_3_player_4_item_0,team_3_player_4_item_1,team_3_player_4_item_2,team_3_player_4_item_3,team_3_player_4_item_4,team_3_player_4_item_5,winner
0,961,8,2294,13,6,1,5,10,142,2278,...,115,566,7876,65,63,170,36,73,73,radiant
1,849,-1,1261,11,4,1,3,6,84,1103,...,8,2110,4010,180,43,-1,36,-1,-1,dire
2,1030,4,2241,13,6,1,3,9,159,1015,...,160,205,10135,168,75,63,236,28,11,dire
3,1044,-10,-6837,12,3,2,3,4,102,1091,...,14,754,4299,261,92,43,593,36,180,dire
4,897,21,13149,10,5,0,3,10,90,1752,...,50,299,4404,92,240,36,67,16,16,radiant


In [24]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dataset['winner'] = label_encoder.fit_transform(dataset['winner'])

In [25]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [27]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [28]:
ann = tf.keras.models.Sequential()

In [29]:
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [30]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
1059/1059 [==============================] - 4s 2ms/step - loss: 0.5471 - accuracy: 0.7007
Epoch 2/100
1059/1059 [==============================] - 2s 2ms/step - loss: 0.4620 - accuracy: 0.7568
Epoch 3/100
1059/1059 [==============================] - 3s 2ms/step - loss: 0.3961 - accuracy: 0.7983
Epoch 4/100
1059/1059 [==============================] - 2s 2ms/step - loss: 0.3447 - accuracy: 0.8252
Epoch 5/100
1059/1059 [==============================] - 2s 2ms/step - loss: 0.3070 - accuracy: 0.8478
Epoch 6/100
1059/1059 [==============================] - 2s 2ms/step - loss: 0.2754 - accuracy: 0.8645
Epoch 7/100
1059/1059 [==============================] - 2s 2ms/step - loss: 0.2547 - accuracy: 0.8754
Epoch 8/100
1059/1059 [==============================] - 2s 2ms/step - loss: 0.2327 - accuracy: 0.8850
Epoch 9/100
1059/1059 [==============================] - 2s 2ms/step - loss: 0.2175 - accuracy: 0.8937
Epoch 10/100
1059/1059 [==============================] - 3s 3ms/step - l

In [32]:
y_test

27628    0
28505    0
35593    1
30933    0
4484     1
        ..
3715     1
7791     1
2462     0
18230    0
24985    1
Name: winner, Length: 3764, dtype: int64

In [33]:
y_pred = ann.predict(X_test)
print(y_pred)

118/118 [==============================] - 0s 1ms/step
[[5.8413834e-25]
 [5.6271098e-04]
 [9.9779785e-01]
 ...
 [6.1205618e-21]
 [3.0757812e-07]
 [9.9950004e-01]]


In [34]:
y_pred = y_pred > 0.5
y_pred

array([[False],
       [False],
       [ True],
       ...,
       [False],
       [False],
       [ True]])

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(cm, acc)

[[1481  272]
 [ 242 1769]] 0.8634431455897981


In [36]:
import pickle

with open('ann.pkl','wb') as f:
    pickle.dump(ann, f)
with open('standart_scaler.pkl','wb') as f:
    pickle.dump(sc, f)

In [37]:
# import pymongo

# MONGO_CLIENT = pymongo.MongoClient("mongodb://192.168.1.7:27017/")
# SDA_DB = MONGO_CLIENT["steam_dota_api"]
# MLRS_COL = SDA_DB["multi_real_time_stats"]

In [38]:
# # matches = MLRS_COL.find({"_id": ObjectId("65b5e65abb22a1e961d58c20")})
# matches = MLRS_COL.find({"match.match_id": "7557978739"})
# matches_for_pd = []

# for entry in matches:
#     try:
#         tmp = {}

#         tmp["timestamp"] = entry["match"]["timestamp"]
#         tmp["game_time"] = entry["match"]["game_time"]

#         tmp['score'] = entry["teams"][0]["score"] - entry["teams"][1]["score"]
#         tmp['net_worth'] = entry["teams"][0]["net_worth"] - entry["teams"][1]["net_worth"]

#         for team in entry["teams"]:
#             for p_idx, player in enumerate(team["players"]):
#                 # tmp["team_{}_player_{}_heroid".format(team["team_number"], p_idx)] = player['heroid']
#                 tmp["team_{}_player_{}_level".format(team["team_number"], p_idx)] = player['level']
#                 tmp["team_{}_player_{}_kill_count".format(team["team_number"], p_idx)] = player['kill_count']
#                 tmp["team_{}_player_{}_death_count".format(team["team_number"], p_idx)] = player['death_count']
#                 tmp["team_{}_player_{}_assists_count".format(team["team_number"], p_idx)] = player['assists_count']
#                 tmp["team_{}_player_{}_denies_count".format(team["team_number"], p_idx)] = player['denies_count']
#                 tmp["team_{}_player_{}_lh_count".format(team["team_number"], p_idx)] = player['lh_count']
#                 tmp["team_{}_player_{}_gold".format(team["team_number"], p_idx)] = player['gold']
#                 tmp["team_{}_player_{}_net_worth".format(team["team_number"], p_idx)] = player['net_worth']

#                 for i_idx, item in enumerate(player['items']):
#                     tmp["team_{}_player_{}_item_{}".format(team["team_number"], p_idx, i_idx)] = item

#         for gg_idx, graph_gold in enumerate(entry['graph_data']['graph_gold']):
#             tmp['graph_gold_{}'.format(gg_idx)] = graph_gold

#         for i, building in enumerate(entry["buildings"]):
#             tmp["building_{}".format(i)] = int(building["destroyed"])

#         matches_for_pd.append(tmp)
#     except Exception as e:
#         MLRS_COL.delete_one({"_id": entry['_id']})
#         print(e, entry['_id'])

In [39]:
# df = pd.DataFrame(matches_for_pd)
# df.info()

In [40]:
# X = df.iloc[:, :].values

In [41]:
# X_ss = sc.transform(X)

In [42]:
# y_pred = ann.predict(X_ss)
# for y in y_pred:
#     if y[0] > 0.5:
#         print('radiant', round(y[0], 2))
#     else:
#         print('dire', round(1- y[0], 2))
#     # print(round(1 - y[0], 4), '\t', round(y[0], 4))
#     # print(y[0], 1 - y[0])